In [1]:
import time
import unittest

import casadi
import example_robot_data as robex
import numpy as np
import pinocchio as pin
from numpy.linalg import norm
from pinocchio import casadi as cpin
from scipy.optimize import fmin_bfgs

from utils.meshcat_viewer_wrapper import MeshcatVisualizer

In [2]:
robot = robex.load("talos_legs")
# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
viz.viewer.jupyter_cell()

/Users/zhangjixiang/Code/jnrh2023/myenv/lib/python3.11/site-packages/cmeel.prefix/lib/python3.11/site-packages/example_robot_data/robots_loader.py:319: UserWarning: Deprecated member. Use Frame.parentJoint instead.
  if f.parent < legMaxId:


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
from pinocchio.utils import eye

model = robot.model
left_jID = model.getJointId("leg_left_6_joint")
right_jID = model.getJointId("leg_right_6_joint")
model.addFrame(pin.Frame('left_contact_point_1', left_jID, pin.SE3(
    eye(3), np.matrix([.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_2', left_jID, pin.SE3(
    eye(3), np.matrix([-.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_3', left_jID, pin.SE3(
    eye(3), np.matrix([-.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_4', left_jID, pin.SE3(
    eye(3), np.matrix([.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_1', right_jID, pin.SE3(
    eye(3), np.matrix([.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_2', right_jID, pin.SE3(
    eye(3), np.matrix([-.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_3', right_jID, pin.SE3(
    eye(3), np.matrix([-.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_4', right_jID, pin.SE3(
    eye(3), np.matrix([.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
data = model.createData()

In [4]:
contact_names = ["left_contact_point_1", "left_contact_point_2", "left_contact_point_3", "left_contact_point_4",
                 "right_contact_point_1", "right_contact_point_2", "right_contact_point_3", "right_contact_point_4"]
# Add a vizualization for the target
for i in range(8):
    cID = "world/" + contact_names[i]
    viz.addSphere(cID, 0.02, [1.0, 0.2, 0.2, 0.5])
    pin.framesForwardKinematics(model, data, robot.q0)
    viz.applyConfiguration(cID, data.oMf[model.getFrameId(contact_names[i])])

In [5]:
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

nq = model.nq
nv = model.nv
cq = casadi.SX.sym("q", nq, 1)
cv = casadi.SX.sym("v", nv, 1)

# Compute kinematics casadi graphs
cpin.forwardKinematics(cmodel, cdata, cq)
cpin.updateFramePlacements(cmodel, cdata)
# Compute com casadi graphs
cpin.centerOfMass(cmodel, cdata, cq)
# Compute centroidal momentum casadi graphs
cpin.ccrba(cmodel, cdata, cq, cv)
print("Get Graphs Done.")

Get Graphs Done.


In [6]:
# Sym graph for the integration operation q,dq -> q(+)dq = model.integrate(q,dq)
cintegrate = casadi.Function(
    "integrate",
    [cq, cv],
    [cpin.integrate(cmodel, cq, cv)]
)

In [7]:
contacts_fk_list = []
for i in range(8):
    endEffector_ID = model.getFrameId(contact_names[i])
    contacts_fk_list.append(casadi.Function(
        contact_names[i]+"Fun",
        [cq],
        [cdata.oMf[endEffector_ID].translation]
    ))

In [8]:
h = casadi.Function(
    "hFun",
    [cq, cv],
    [cdata.hg.angular]
)

In [9]:
com = casadi.Function(
    "comFun",
    [cq],
    [cdata.com[0]]
)

In [10]:
print("com = ", com(robot.q0))
print("h = ", h(robot.q0, np.zeros(nv)))
for i in range(8):
    print(contact_names[i], contacts_fk_list[i](robot.q0))

com =  [0.0034147, 0.00228237, 0.618288]
h =  [0, 0, 0]
left_contact_point_1 [0.091153, 0.144829, 0.00689549]
left_contact_point_2 [-0.108847, 0.144829, 0.00689549]
left_contact_point_3 [-0.108847, 0.0248293, 0.00710045]
left_contact_point_4 [0.091153, 0.0248293, 0.00710045]
right_contact_point_1 [0.091153, -0.0251709, 0.00689549]
right_contact_point_2 [-0.108847, -0.0251709, 0.00689549]
right_contact_point_3 [-0.108847, -0.145171, 0.00710045]
right_contact_point_4 [0.091153, -0.145171, 0.00710045]


In [11]:
N = 31
opti = casadi.Opti()
var_dq = [opti.variable(robot.nv) for t in range(N)]
var_q = [cintegrate(robot.q0, dq) for dq in var_dq]
var_v = [opti.variable(robot.nv) for t in range(N)]
var_t = [opti.variable(1) for t in range(N)]
var_r = [com(q) for q in var_q]
var_dr = [opti.variable(3) for t in range(N)]
var_ddr = [opti.variable(3) for t in range(N)]
var_c = [[contacts_fk_list[i](q) for i in range(8)]
         for q in var_q]   # len(var_c) == N
var_F = [[opti.variable(3) for i in range(8)]
         for t in range(N)]  # len(var_F) == N
var_h = [h(var_q[t], var_v[t]) for t in range(N)]
var_dh = [opti.variable(3) for t in range(N)]